In [1]:
import os
os.chdir('..')

In [2]:
from ddi_kt_2024.mol.gnn import GNN
from ddi_kt_2024.smiles.smiles_bert import SMILESEmbedding
from ddi_kt_2024.utils import load_pkl
from ddi_kt_2024.mol.preprocess import mapped_smiles_reader, candidate_smiles
from ddi_kt_2024.mol.mol_dataset import MolDataset

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-05 02:19:58,232 - INFO - Enabling RDKit 2023.09.5 jupyter extensions


In [3]:
all_candidates_train = load_pkl('cache/pkl/v1/candidates.train.pkl')
all_candidates_test = load_pkl('cache/pkl/v1/candidates.test.pkl')
mapped_smiles = mapped_smiles_reader('cache/mapped_drugs/DDI/all_mapped.txt')
x_train, y_train = candidate_smiles(all_candidates_train, mapped_smiles)
x_test, y_test = candidate_smiles(all_candidates_test, mapped_smiles)
# dataset_train_mol1 = MolDataset(x_train, element=1)
# dataset_train_mol2 = MolDataset(x_train, element=2)
# dataset_test_mol1 = MolDataset(x_test, element=1)
dataset_test_mol2 = MolDataset(x_test, element=2)

Converting SMILES to PyG: 100%|██████████| 5716/5716 [00:03<00:00, 1535.75it/s]


In [13]:
model = SMILESEmbedding()

In [14]:
mol = dataset_test_mol2[121]
mol

Data(x=[1, 10], edge_index=[2, 2], edge_attr=[2, 4], mol='None', smiles='None')

In [15]:
model(mol).shape

torch.Size([1, 600])